In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from scipy.spatial import KDTree

G_NORMAL = 0.05
G_MICRO = 0

class Particle:
    def __init__(self, unique_id, model):
        self.unique_id = unique_id
        self.model = model
        self.mass = np.random.uniform(0.5, 1.5)
        self.resources = 0
        self.pos = np.array([np.random.uniform(0, 99.999), np.random.uniform(0, 99.999)])
        self.bonds = set()

    def step(self):
        density = 1025 + (1076 - 1025) * (100 - self.pos[1]) / 100
        if self.model.G != G_MICRO:
            self.pos[1] += -self.model.G * self.mass
        step_size = 0.5 / np.sqrt(density / 1025)
        self.pos += np.random.uniform(-step_size, step_size, 2)
        self.pos = np.clip(self.pos, 0, 99.999)
        base_density = max(0, 10 - self.pos[1] / 10) if self.model.G != G_MICRO else 5
        self.resources += base_density * 0.1 * (density / 1025)

class MolecularModel:
    def __init__(self, N_particles, G):
        self.G = G
        self.particles = [Particle(i, self) for i in range(N_particles)]
        self.particle_dict = {p.unique_id: p for p in self.particles}
        self.bond_events = 0
        self.removed_particles = set()

    def step(self):
        for p in self.particles:
            p.step()
        positions = np.array([p.pos for p in self.particles])
        if len(positions) > 0:
            tree = KDTree(positions)
            for i, particle in enumerate(self.particles):
                if particle.unique_id in self.removed_particles:
                    continue
                indices = tree.query_ball_point(particle.pos, 1.0)
                for j in indices:
                    if i == j or self.particles[j].unique_id in self.removed_particles:
                        continue
                    other = self.particles[j]
                    # Pressure bonus: lower threshold at depth
                    bond_threshold = 4 if particle.pos[1] < 10 else 5
                    if (particle.resources > bond_threshold and other.resources > bond_threshold and 
                            other.unique_id not in particle.bonds):
                        particle.bonds.add(other.unique_id)
                        other.bonds.add(particle.unique_id)
                        particle.resources -= bond_threshold
                        other.resources -= bond_threshold
                        self.bond_events += 1
                        break
                    elif particle.resources > 10 and other.resources < 3:
                        particle.resources += other.resources
                        particle.mass += other.mass * 0.5
                        self.removed_particles.add(other.unique_id)
                        break
            if self.removed_particles:
                for p in self.particles:
                    p.bonds -= self.removed_particles
                self.particles = [p for p in self.particles if p.unique_id not in self.removed_particles]
                for uid in self.removed_particles:
                    self.particle_dict.pop(uid, None)
                self.removed_particles.clear()

def analyze_chains(particles):
    chain_lengths = []
    visited = set()
    for p in particles:
        if p.unique_id not in visited and p.bonds:
            chain = []
            queue = deque([p.unique_id])
            visited.add(p.unique_id)
            while queue:
                current_id = queue.popleft()
                chain.append(current_id)
                current = next((p for p in particles if p.unique_id == current_id), None)
                if current:
                    for bond_id in current.bonds:
                        if bond_id not in visited:
                            visited.add(bond_id)
                            queue.append(bond_id)
            chain_lengths.append(len(chain))
    return chain_lengths

def run_simulation(gravity, label, steps=1000, runs=5):
    all_stats = []
    for run in range(runs):
        model = MolecularModel(200, gravity)
        for step in range(steps + 1):
            model.step()
        chain_lengths = analyze_chains(model.particles)
        stats = {
            "bond_events": model.bond_events,
            "population": len(model.particles),
            "avg_resources": np.mean([p.resources for p in model.particles]) if model.particles else 0,
            "avg_mass": np.mean([p.mass for p in model.particles]) if model.particles else 0,
            "avg_chain_length": np.mean(chain_lengths) if chain_lengths else 0,
            "max_chain_length": max(chain_lengths) if chain_lengths else 0
        }
        all_stats.append(stats)
    
    # Average across runs
    avg_stats = {key: np.mean([s[key] for s in all_stats]) for key in all_stats[0].keys()}
    print(f"{label} (Averaged over {runs} runs):")
    print(f"  Bond Events: {avg_stats['bond_events']:.0f}")
    print(f"  Population: {avg_stats['population']:.0f}")
    print(f"  Avg Resources: {avg_stats['avg_resources']:.2f}")
    print(f"  Avg Mass: {avg_stats['avg_mass']:.2f}")
    print(f"  Avg Chain Length: {avg_stats['avg_chain_length']:.2f}")
    print(f"  Max Chain Length: {avg_stats['max_chain_length']:.0f}")
    return avg_stats

# Run simulations
print("Normal Gravity")
normal_results = run_simulation(G_NORMAL, "Normal Gravity")
print("\nMicrogravity")
micro_results = run_simulation(G_MICRO, "Microgravity")

Normal Gravity
Normal Gravity (Averaged over 5 runs):
  Bond Events: 837
  Population: 200
  Avg Resources: 692.04
  Avg Mass: 0.98
  Avg Chain Length: 130.57
  Max Chain Length: 193

Microgravity
Microgravity (Averaged over 5 runs):
  Bond Events: 381
  Population: 200
  Avg Resources: 494.09
  Avg Mass: 1.00
  Avg Chain Length: 66.50
  Max Chain Length: 183
